In [20]:
import pandas as pd
import re
import datetime as dt
import numpy as np

Esercizio 1.

Normalize the loan_lenders table. In the normalized table, each row must have one loan_id and one lender.

In [7]:
ll=pd.read_csv('C:\\Users\\symo9\\Desktop\\FoCS\\progetto\\loans_lenders.csv')

In [10]:
a=pd.DataFrame()
def dividi(stringa):
    return re.split(',',stringa)
ll['lista']=ll.apply(lambda row: dividi(row['lenders']), axis=1)

In [11]:
lenders=[nome for lista in ll['lista'] for nome in lista]


In [12]:
ll['lunghezza']=[len(x) for x in ll['lista']]

In [13]:
listone=[]
i=0
for i in range(0,len(ll)):
    listone.append([ll['loan_id'][i]]*ll['lunghezza'][i])

In [14]:
loan_id=[y for x in listone for y in x]

In [15]:
lo_le=pd.DataFrame({'loan_id':loan_id, 'lenders':lenders})

In [16]:
lo_le.head()

,loan_id,lenders
0,483693,muc888
1,483693,sam4326
2,483693,camaran3922
3,483693,lachheb1865
4,483693,rebecca3499


Esercizio 2.

For each loan, add a column duration corresponding to the number of days between the disburse time and the planned expiration time. If any of those two dates is missing, also the duration must be missing.


In [4]:
l=pd.read_csv('C:\\Users\\symo9\\Desktop\\FoCS\\progetto\\loans.csv')

In [22]:
l['planned_expiration_time']=pd.to_datetime(l['planned_expiration_time'])
l['disburse_time']=pd.to_datetime(l['disburse_time'])
l['duration']=abs(l['planned_expiration_time']-l['disburse_time'])
l[['loan_id','duration']].head()


,loan_id,duration
0,657307,53 days 19:30:06
1,657259,96 days 14:25:07
2,658010,37 days 13:10:05
3,659347,34 days 19:10:02
4,656933,57 days 22:10:02


Esercizio 3. 

Find the lenders that have funded at least twice.


In [17]:
lenders_twice=lo_le.groupby('lenders').count()
lenders_twice2=lenders_twice[lenders_twice['loan_id']>1]
lenders_twice2=lenders_twice2.reset_index()
lenders_twice2['lenders'].head()

0                000
1              00000
2               0002
3     0101craign0101
4            0132575
Name: lenders, dtype: object

Esercizio 4.

For each country, compute how many loans have involved that country as borrowers.


In [41]:
paesi=l.groupby('country_name').size()
paesi.head()

country_name
Afghanistan                               2337
Albania                                   3075
Armenia                                  13952
Azerbaijan                               10172
Belize                                     218
Benin                                     5946
Bhutan                                       2
Bolivia                                  25250
Bosnia and Herzegovina                     608
Botswana                                     1
Brazil                                     482
Bulgaria                                   296
Burkina Faso                              3489
Burundi                                   1727
Cambodia                                 79701
Cameroon                                  5085
Canada                                       1
Chad                                        61
Chile                                      892
China                                      134
Colombia                                 33675


Esercizio 5.

For each country, compute the overall amount of money borrowed.


In [18]:
l.groupby('country_name')['loan_amount'].sum().head()

country_name
Afghanistan     1967950.0
Albania         4307350.0
Armenia        22950475.0
Azerbaijan     14784625.0
Belize           150175.0
Name: loan_amount, dtype: float64

Esercizio 6.

Like the previous point, but expressed as a percentage of the overall amount lent.


In [25]:
stati=l.groupby('country_name')[['loan_amount']].sum()
totale=sum(stati['loan_amount'])
stati['loan_percentage']=(stati['loan_amount']/totale)*100
stati=stati['loan_percentage']

In [26]:
stati.head()

country_name
Afghanistan    0.166573
Albania        0.364586
Armenia        1.942589
Azerbaijan     1.251410
Belize         0.012711
Name: loan_percentage, dtype: float64

Esercizio 7.

Like the three previous points, but split for each year (with respect to disburse time).


In [28]:
def anno(data):
    a=pd.to_datetime(data)
    return a.year


In [29]:
l['year']=l.apply(lambda row: anno(row['disburse_time']), axis=1)

In [30]:
paesi2=l.groupby(['year','country_name']).size()

In [12]:
l.groupby(['year','country_name'])['loan_amount'].sum()

year    country_name                    
2005.0  Bulgaria                               1850.0
        Cambodia                               1625.0
        Ecuador                                5000.0
        Gaza                                   5000.0
        Honduras                              33300.0
        Kenya                                 21350.0
        Nicaragua                              3300.0
        Senegal                                3225.0
        Tanzania                               2300.0
        Uganda                                25900.0
2006.0  Azerbaijan                            15700.0
        Bulgaria                              84650.0
        Cambodia                              41950.0
        Ecuador                              151175.0
        Ghana                                 23425.0
        Honduras                              94325.0
        Kenya                                222800.0
        Mexico                           

In [31]:
tot=l.groupby('year')[['loan_amount']].sum()
tot.head()

,loan_amount
year,
2005.0,102850.0
2006.0,1376575.0
2007.0,15446525.0
2008.0,39423050.0
2009.0,59689475.0


In [34]:
stati2=l.groupby(['year','country_name'])[['loan_amount']].sum()
stati2=stati2.reset_index() 
nuovo=pd.merge(stati2, tot, on='year')
nuovo['percentage']=(nuovo['loan_amount_x']/nuovo['loan_amount_y'])*100
nuovo=nuovo[['year','country_name','percentage']]
nuovo.head()

,year,country_name,percentage
0,2005.0,Bulgaria,1.798736
1,2005.0,Cambodia,1.579971
2,2005.0,Ecuador,4.861449
3,2005.0,Gaza,4.861449
4,2005.0,Honduras,32.377248


Esercizio 8.

For each lender, compute the overall amount of money lent. For each loan that has more than one lender, you must assume that all lenders contributed the same amount.


In [5]:
l['single_fund']=l['funded_amount']/l['num_lenders_total']

In [8]:
new=pd.merge(l, ll, on='loan_id')[['loan_id','lenders','single_fund']]

In [17]:
new2=pd.merge(new,lo_le,on='loan_id')[['loan_id', 'single_fund', 'lenders_y']]

In [18]:
money_lent=new2.groupby('lenders_y')[['single_fund']].sum()

In [19]:
money_lent.head()

,single_fund
lenders_y,
000,1485.309656
00000,1249.127691
0002,2201.180463
00mike00,38.461538
0101craign0101,2424.088932


Esercizio 9.

For each country, compute the difference between the overall amount of money lent and the overall amount of money borrowed. Since the country of the lender is often unknown, you can assume that the true distribution among the countries is the same as the one computed from the rows where the country is known.


In [35]:
lend=pd.read_csv('C:\\Users\\symo9\\Desktop\\FoCS\\progetto\\lenders.csv')

In [36]:
vuoto=lend[lend['country_code'].isnull()==True]

In [37]:
pieno=lend[lend['country_code'].isnull()==False]

In [70]:
conteggio=pieno.groupby(['country_code']).size()

In [71]:
conteggio=conteggio.reset_index()

In [78]:
stati=[x for x in conteggio['country_code']]

In [80]:
volte=[x for x in conteggio[0]]

In [82]:
listone2=[]
for i in range(0,len(stati)):
    listone2.append([stati[i]]*volte[i])

In [94]:
listone2

[['AD',
  'AD',
  'AD',
  'AD',
  'AD',
  'AD',
  'AD',
  'AD',
  'AD',
  'AD',
  'AD',
  'AD',
  'AD',
  'AD',
  'AD'],
 ['AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',
  'AE',

In [93]:
end=[y for x in listone2 for y in x]

In [84]:
import random
vect=[]
for i in range(0,len(vuoto)):
    vect.append(random.sample(end,1)[0])

In [85]:
vuoto['country_code']=vect

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [46]:
new_lenders=pieno.append(vuoto)

In [49]:
new_lenders.rename(columns={'permanent_name': 'lenders_y'}, inplace=True)


In [50]:
prestatori=pd.merge(new2, new_lenders, on='lenders_y')[['loan_id', 'lenders_y', 'country_code', 'single_fund']]

In [51]:
somma_prestati=prestatori.groupby('country_code')[['single_fund']].sum()

In [54]:
somma_ricevuti=l.groupby('country_code')[['funded_amount']].sum()

In [60]:
entrambi=pd.merge(somma_prestati, somma_ricevuti, on = 'country_code', how='left')

In [61]:
entrambi.head()

,single_fund,funded_amount
country_code,,
AD,101.190476,NaN
AE,103510.202092,NaN
AF,6227.549291,1964650.0
AG,291.569913,NaN
AL,1371.273615,3992475.0


In [62]:
prest=[]
for i in range(0, len(entrambi)):
    if pd.isna(entrambi.iloc[i]['funded_amount'])==True:
        prest.append(0)
    else:
        prest.append(entrambi.iloc[i]['funded_amount'])
entrambi['funded_amount']=prest       

In [63]:
entrambi['differenza']=entrambi['single_fund']-entrambi['funded_amount']

In [64]:
entrambi.head()

,single_fund,funded_amount,differenza
country_code,,,
AD,101.190476,0.0,1.011905e+02
AE,103510.202092,0.0,1.035102e+05
AF,6227.549291,1964650.0,-1.958422e+06
AG,291.569913,0.0,2.915699e+02
AL,1371.273615,3992475.0,-3.991104e+06


Esercizio 10.

Which country has the highest ratio between the difference computed at the previous point and the population?


In [46]:
country_stats=pd.read_csv('C:\\Users\\symo9\\Desktop\\FoCS\\progetto\\country_stats.csv')

In [59]:
iniziale=pd.merge(country_stats, entrambi, on='country_code')[['country_name', 'population_below_poverty_line', 'population', 'differenza']]

In [61]:
primo_rapporto=iniziale
primo_rapporto['first_ratio']=primo_rapporto['differenza']/primo_rapporto['population']

In [66]:
fine_primo=primo_rapporto[['country_name', 'first_ratio']]

In [84]:
vincitore=fine_primo.iloc[fine_primo['first_ratio'].idxmax()]

In [85]:
vincitore

country_name    Luxembourg
first_ratio       0.227408
Name: 153, dtype: object

Esercizio 11.

Which country has the highest ratio between the difference computed at point 9 and the population that is not below the poverty line?


In [92]:
secondo_rapporto=iniziale
secondo_rapporto['second_ratio']=secondo_rapporto['differenza']/secondo_rapporto['population_below_poverty_line']

In [94]:
fine_secondo=secondo_rapporto[['country_name', 'second_ratio']]

In [100]:
vincitore2=fine_secondo.iloc[fine_secondo['second_ratio'].idxmax()]

In [101]:
vincitore2

country_name    United States
second_ratio       2.7817e+06
Name: 6, dtype: object

Esercizio 12.

For each year, compute the total amount of loans. Each loan that has planned expiration time and disburse time in different years must have its amount distributed proportionally to the number of days in each year. For example, a loan with disburse time December 1st, 2016, planned expiration time January 30th 2018, and amount 5000USD has an amount of 5000USD * 31 / (31+365+30) = 363.85 for 2016, 5000USD * 365 / (31+365+30) = 4284.04 for 2017, and 5000USD * 30 / (31+365+30) = 352.11 for 2018.


In [6]:
l['year2']= l.apply(lambda row: anno(row['planned_expiration_time']), axis=1)

In [7]:
l1=l[(l['year2']-l['year'])>=1]
l1=l1.reset_index()
l2=l[(l['year2']-l['year'])==0]
l2=l2.reset_index()
ammonti=[x for x in l2['loan_amount']]
years=[x for x in l2['year']]
c=pd.DataFrame({'loan':ammonti, 'anni':years})

In [45]:
lista_finale_pesi=[]
lista_finale_anni=[]

In [46]:
for i in range(0,len(l1)):
    diff=l1['year2'][i]-l1['year'][i]
    k=0
    lista2=[]
    for k in range(0,int(diff)+1):
        lista2.append(l1['year'][i]+k)
        if k==int(diff) and int(diff)>1:
            lista1=[x for x in range(0,int(diff)+1)]
            for j in range(1, k):
                lista1[j]=365
        else:
            lista1=[x for x in range(0,2)]
    lista1[0]=365-(pd.to_datetime(l1['disburse_time'][i])).dayofyear
    lista1[-1]=(pd.to_datetime(l1['planned_expiration_time'][i])).dayofyear
    pesi=[x*l1['loan_amount'][i]/sum(lista1) for x in lista1]
    for h in range(0,len(pesi)):
        lista_finale_pesi.append(pesi[h])
        lista_finale_anni.append(lista2[h])    

In [47]:
b=pd.DataFrame({'loan':lista_finale_pesi, 'anni':lista_finale_anni})

In [48]:
finale=c.append(b)

In [120]:
finale.groupby('anni')[['loan']].sum()

,loan
anni,
2011.0,6.299119e+05
2012.0,1.131230e+08
2013.0,1.250890e+08
2014.0,1.531511e+08
2015.0,1.597600e+08
2016.0,1.587487e+08
2017.0,1.721008e+08
2018.0,5.705071e+06
